In [1]:
from symred.symplectic_form import PauliwordOp, StabilizerOp, AnsatzOp, ObservableOp, QuantumState
from symred.S3_projection import (S3_projection, 
                                  gf2_gaus_elim, 
                                  gf2_basis_for_gf2_rref, 
                                  QubitTapering, 
                                  CS_VQE, 
                                  CheatS_VQE)
from symred.build_model import build_molecule_for_projection
from symred.utils import exact_gs_energy, quasi_model
import json
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator
from pyscf.cc.addons import spatial2spin


import warnings
warnings.filterwarnings("ignore")

# Build the molecule

In [2]:
with open('data/molecule_data.json', 'r') as jfile:
    molecule_geometries = json.load(jfile)
print(molecule_geometries.keys())

dict_keys(['H2_3-21G_SINGLET', 'H6_STO-3G_SINGLET', 'H2_6-31G_SINGLET', 'H2_6-311G_SINGLET', 'H3+_STO-3G_SINGLET', 'H3+_3-21G_SINGLET', 'HeH+_3-21G_SINGLET', 'HeH+_6-311G_SINGLET', 'H2O_STO-3G_SINGLET', 'BeH+_STO-3G_SINGLET', 'LiH_STO-3G_SINGLET', 'CH+_STO-3G_SINGLET', 'HF_STO-3G_SINGLET', 'B+_STO-3G_SINGLET', 'B_STO-3G_DOUBLET', 'N_STO-3G_QUARTET', 'OH-_STO-3G_SINGLET', 'O_STO-3G_TRIPLET', 'CH2_STO-3G_TRIPLET', 'BeH2_STO-3G_SINGLET', 'Be_STO-3G_SINGLET', 'C_STO-3G_TRIPLET', 'NH_STO-3G_SINGLET', 'Ne_STO-3G_SINGLET', 'F_STO-3G_DOUBLET', 'Li_STO-3G_DOUBLET', 'BH_STO-3G_SINGLET', 'NeH+_STO-3G_SINGLET', 'NH2+_STO-3G_SINGLET', 'BH2+_STO-3G_SINGLET', 'HCl_STO-3G_SINGLET', 'H4_STO-3G_SINGLET', 'NH3_STO-3G_SINGLET', 'F2_STO-3G_SINGLET', 'HCN_STO-3G_SINGLET', 'CH4_STO-3G_SINGLET', 'CH3NHCH3_STO-3G_SINGLET', 'CH3CH2NH2_STO-3G_SINGLET', 'CH3CH2OH_STO-3G_SINGLET', 'CH3OH_STO-3G_SINGLET', 'C2H6_STO-3G_SINGLET', 'CH3CN_STO-3G_SINGLET', 'CH3CHO_STO-3G_SINGLET', 'CH3CHOHCH3_STO-3G_SINGLET', 'CHONH2_ST

In [46]:
# Set molecule parameters
speciesname = 'SiH4_STO-3G_SINGLET'
mol_data = molecule_geometries[speciesname]
if 'name' in mol_data:
    print(mol_data['name'])
    
atoms = mol_data['atoms']
coords = mol_data['coords']
basis = mol_data['basis']
multiplicity = mol_data['multiplicity']
charge = mol_data['charge']
geometry = list(zip(atoms, coords))

xyz_file = str(len(atoms))+'\n '

for atom, coords in geometry:
    xyz_file += '\n'+atom+'\t'
    xyz_file += '\t'.join(list(map(str, coords)))

print(xyz_file)
xyz_file

silane
5
 
Si	0.0	0.0	0.0
H	0.8544	0.8544	0.8544
H	-0.8544	-0.8544	0.8544
H	-0.8544	0.8544	-0.8544
H	0.8544	-0.8544	-0.8544


'5\n \nSi\t0.0\t0.0\t0.0\nH\t0.8544\t0.8544\t0.8544\nH\t-0.8544\t-0.8544\t0.8544\nH\t-0.8544\t0.8544\t-0.8544\nH\t0.8544\t-0.8544\t-0.8544'

In [47]:
from symred.chem import PySCFDriver

convergence = 1e6; max_hf_cycles=100_000; ram = 8_000
run_mp2  = True; run_cisd = False; run_ccsd = True; run_fci  = True

pyscf_obj = PySCFDriver(xyz_file,
                        basis,
                        #convergence=convergence,
                        charge=charge,
                        #max_ram_memory=ram,
                        #max_hf_cycles=max_hf_cycles,                   
                        run_mp2=run_mp2,
                        run_cisd=run_cisd,
                        run_ccsd=run_ccsd,
                        run_fci=run_fci)

In [48]:
pyscf_obj.run_pyscf()
print('Converged?', pyscf_obj.pyscf_ccsd.converged)

Converged? True


In [49]:
from symred.chem import FermionicHamilt
import openfermion as of

H_ferm = FermionicHamilt(pyscf_obj.pyscf_hf)
H_ferm.build_operator()
ham_fermionic = of.get_fermion_operator(H_ferm.fermionic_molecular_hamiltonian)
ham_jw = of.jordan_wigner(ham_fermionic)
n_qubits = H_ferm.n_qubits

def QubitOperator_to_dict(op, num_qubits):
    assert(type(op) == of.QubitOperator)
    op_dict = {}
    term_dict = op.terms
    terms = list(term_dict.keys())

    for t in terms:    
        letters = ['I' for i in range(num_qubits)]
        for i in t:
            letters[i[0]] = i[1]
        p_string = ''.join(letters)        
        op_dict[p_string] = term_dict[t]
         
    return op_dict

ham_dict = QubitOperator_to_dict(ham_jw, n_qubits)
ham = ObservableOp({op:coeff for op,coeff in ham_dict.items() if coeff.imag==0})

print('Jordan-Wigner Hamiltonian:\n\n', ham)

Jordan-Wigner Hamiltonian:

 -177.8210051991 IIIIIIIIIIIIIIIIIIIIIIIIII +
40.4589841171 ZIIIIIIIIIIIIIIIIIIIIIIIII +
0.4019847228 YZYIIIIIIIIIIIIIIIIIIIIIII +
0.4019847228 XZXIIIIIIIIIIIIIIIIIIIIIII +
-0.0770577176 YZZZZZZZZZYIIIIIIIIIIIIIII +
-0.0770577176 XZZZZZZZZZXIIIIIIIIIIIIIII +
-0.0790199818 YZZZZZZZZZZZZZZZZZYIIIIIII +
-0.0790199818 XZZZZZZZZZZZZZZZZZXIIIIIII +
40.4589841171 IZIIIIIIIIIIIIIIIIIIIIIIII +
0.4019847228 IYZYIIIIIIIIIIIIIIIIIIIIII +
0.4019847228 IXZXIIIIIIIIIIIIIIIIIIIIII +
-0.0770577176 IYZZZZZZZZZYIIIIIIIIIIIIII +
-0.0770577176 IXZZZZZZZZZXIIIIIIIIIIIIII +
-0.0790199818 IYZZZZZZZZZZZZZZZZZYIIIIII +
-0.0790199818 IXZZZZZZZZZZZZZZZZZXIIIIII +
6.6219245715 IIZIIIIIIIIIIIIIIIIIIIIIII +
0.3312871979 IIYZZZZZZZYIIIIIIIIIIIIIII +
0.3312871979 IIXZZZZZZZXIIIIIIIIIIIIIII +
0.3236298922 IIYZZZZZZZZZZZZZZZYIIIIIII +
0.3236298922 IIXZZZZZZZZZZZZZZZXIIIIIII +
6.6219245715 IIIZIIIIIIIIIIIIIIIIIIIIII +
0.3312871979 IIIYZZZZZZZYIIIIIIIIIIIIII +
0.3312871979 IIIXZZZZZZZXIIIIIIIII

# Compute the Coupled-Cluster parameters

In [50]:
# singles

t1 = spatial2spin(pyscf_obj.pyscf_ccsd.t1)
no, nv = t1.shape
nmo = no + nv
ccsd_single_amps = np.zeros((nmo, nmo))
ccsd_single_amps[no:,:no] = t1.T

single_amplitudes_list = []
for i, j in zip(*ccsd_single_amps.nonzero()):
    single_amplitudes_list.append([[i, j], ccsd_single_amps[i, j]])

generator_t1 = FermionOperator()
for (i, j), t_ij in single_amplitudes_list:
    i, j = int(i), int(j)
    generator_t1 += FermionOperator(((i, 1), (j, 0)), t_ij)
    
T1_dict = QubitOperator_to_dict(of.jordan_wigner(generator_t1), n_qubits)
try:
    T1 = PauliwordOp(T1_dict)
    T1 -= T1.conjugate
    T1 = T1.cleanup_zeros()
    T1 = T1.multiply_by_constant(-1j)
    print('Single excitation terms:\n\n', T1)
except:
    print('No single excitations')

Single excitation terms:

 0.0000866914-0.0000000000j IIIIIIIIIIIIIIIIIXZZZZZZZY +
-0.0000866914+0.0000000000j IIIIIIIIIIIIIIIIIYZZZZZZZX +
-0.0022844737+0.0000000000j IIIIIIIIIIIIIIIIIXZZZZZYII +
0.0022844737-0.0000000000j IIIIIIIIIIIIIIIIIYZZZZZXII +
0.0000610397-0.0000000000j IIIIIIIIIIIIIIIIIXZZZYIIII +
-0.0000610397+0.0000000000j IIIIIIIIIIIIIIIIIYZZZXIIII +
0.0000866914-0.0000000000j IIIIIIIIIIIIIIIIXZZZZZZZYI +
-0.0000866914+0.0000000000j IIIIIIIIIIIIIIIIYZZZZZZZXI +
-0.0022844737+0.0000000000j IIIIIIIIIIIIIIIIXZZZZZYIII +
0.0022844737-0.0000000000j IIIIIIIIIIIIIIIIYZZZZZXIII +
0.0000610397-0.0000000000j IIIIIIIIIIIIIIIIXZZZYIIIII +
-0.0000610397+0.0000000000j IIIIIIIIIIIIIIIIYZZZXIIIII +
-0.0022789733+0.0000000000j IIIIIIIIIIIIIIIXZZZZZZZZZY +
0.0022789733-0.0000000000j IIIIIIIIIIIIIIIYZZZZZZZZZX +
-0.0000909591+0.0000000000j IIIIIIIIIIIIIIIXZZZZZZZYII +
0.0000909591-0.0000000000j IIIIIIIIIIIIIIIYZZZZZZZXII +
-0.0001675360+0.0000000000j IIIIIIIIIIIIIIIXZZZZZYIIII +
0.0001675360

In [51]:
# doubles!

t2 = spatial2spin(pyscf_obj.pyscf_ccsd.t2)
no, nv = t2.shape[1:3]
nmo = no + nv
double_amps = np.zeros((nmo, nmo, nmo, nmo))
double_amps[no:,:no,no:,:no] = .5 * t2.transpose(2,0,3,1)

double_amplitudes_list=[]
double_amplitudes = double_amps
for i, j, k, l in zip(*double_amplitudes.nonzero()):
    if not np.isclose(double_amplitudes[i, j, k, l], 0):
        double_amplitudes_list.append([[i, j, k, l],
                                       double_amplitudes[i, j, k, l]])
    
generator_t2 = FermionOperator()

# Add double excitations
for (i, j, k, l), t_ijkl in double_amplitudes_list:
    i, j, k, l = int(i), int(j), int(k), int(l)
    generator_t2 += FermionOperator(((i, 1), (j, 0), (k, 1), (l, 0)), t_ijkl)
#     if anti_hermitian:
#         generator += FermionOperator(((l, 1), (k, 0), (j, 1), (i, 0)),
#                                      -t_ijkl)

T2_dict = QubitOperator_to_dict(of.jordan_wigner(generator_t2), n_qubits)
T2 = PauliwordOp(T2_dict)
T2 -= T2.conjugate
T2 = T2.cleanup_zeros()
T2 = T2.multiply_by_constant(-1j)


print('Double excitation terms:\n\n', T2)

Double excitation terms:

 0.0057846128-0.0000000000j IIIIIIIIIIIIIIIIXXIIIIIIXY +
0.0057846128-0.0000000000j IIIIIIIIIIIIIIIIXXIIIIIIYX +
-0.0057846128+0.0000000000j IIIIIIIIIIIIIIIIXYIIIIIIXX +
0.0057846128-0.0000000000j IIIIIIIIIIIIIIIIXYIIIIIIYY +
-0.0057846128+0.0000000000j IIIIIIIIIIIIIIIIYXIIIIIIXX +
0.0057846128-0.0000000000j IIIIIIIIIIIIIIIIYXIIIIIIYY +
-0.0057846128+0.0000000000j IIIIIIIIIIIIIIIIYYIIIIIIXY +
-0.0057846128+0.0000000000j IIIIIIIIIIIIIIIIYYIIIIIIYX +
-0.0017727264+0.0000000000j IIIIIIIIIIIIIIIIXXIIIIIXYI +
-0.0017727264+0.0000000000j IIIIIIIIIIIIIIIIXXIIIIIYXI +
0.0017727264-0.0000000000j IIIIIIIIIIIIIIIIXYIIIIIXXI +
-0.0017727264+0.0000000000j IIIIIIIIIIIIIIIIXYIIIIIYYI +
0.0017727264-0.0000000000j IIIIIIIIIIIIIIIIYXIIIIIXXI +
-0.0017727264+0.0000000000j IIIIIIIIIIIIIIIIYXIIIIIYYI +
0.0017727264-0.0000000000j IIIIIIIIIIIIIIIIYYIIIIIXYI +
0.0017727264-0.0000000000j IIIIIIIIIIIIIIIIYYIIIIIYXI +
0.0017727264-0.0000000000j IIIIIIIIIIIIIIIIXXIIIIXZZY +
0.0017727264-

The UCCSD operator:

In [52]:
try:
    T_antiherm = T1 + T2.multiply_by_constant(0.5)
except:
    T_antiherm = T2.multiply_by_constant(0.5)

T_antiherm = AnsatzOp(T_antiherm.symp_matrix, T_antiherm.coeff_vec)
print('UCCSD operator:\n\n', T_antiherm)

UCCSD operator:

 0.0000866914 IIIIIIIIIIIIIIIIIXZZZZZZZY +
-0.0000866914 IIIIIIIIIIIIIIIIIYZZZZZZZX +
-0.0022844737 IIIIIIIIIIIIIIIIIXZZZZZYII +
0.0022844737 IIIIIIIIIIIIIIIIIYZZZZZXII +
0.0000610397 IIIIIIIIIIIIIIIIIXZZZYIIII +
-0.0000610397 IIIIIIIIIIIIIIIIIYZZZXIIII +
0.0000866914 IIIIIIIIIIIIIIIIXZZZZZZZYI +
-0.0000866914 IIIIIIIIIIIIIIIIYZZZZZZZXI +
-0.0022844737 IIIIIIIIIIIIIIIIXZZZZZYIII +
0.0022844737 IIIIIIIIIIIIIIIIYZZZZZXIII +
0.0000610397 IIIIIIIIIIIIIIIIXZZZYIIIII +
-0.0000610397 IIIIIIIIIIIIIIIIYZZZXIIIII +
0.0028923064 IIIIIIIIIIIIIIIIXXIIIIIIXY +
0.0028923064 IIIIIIIIIIIIIIIIXXIIIIIIYX +
-0.0028923064 IIIIIIIIIIIIIIIIXYIIIIIIXX +
0.0028923064 IIIIIIIIIIIIIIIIXYIIIIIIYY +
-0.0028923064 IIIIIIIIIIIIIIIIYXIIIIIIXX +
0.0028923064 IIIIIIIIIIIIIIIIYXIIIIIIYY +
-0.0028923064 IIIIIIIIIIIIIIIIYYIIIIIIXY +
-0.0028923064 IIIIIIIIIIIIIIIIYYIIIIIIYX +
-0.0008863632 IIIIIIIIIIIIIIIIXXIIIIIXYI +
-0.0008863632 IIIIIIIIIIIIIIIIXXIIIIIYXI +
0.0008863632 IIIIIIIIIIIIIIIIXYIIIIIXXI +
-0.0

In [53]:
hf_array = H_ferm.hf_comp_basis_state
hf_state = QuantumState([hf_array])
print('Reference state:', hf_state)
print()

op = T_antiherm.copy()
op.coeff_vec = np.sin(1j*op.coeff_vec)

uccsd_state_rough = ((op * hf_state) + hf_state).normalize
print('Rough UCCSD state:\n')
print(uccsd_state_rough)

#print()
#uccsd_state = T_antiherm.exponentiate() * hf_state
#print('UCCSD state:\n')
#print(uccsd_state)

Reference state:  1.0000000000 |11111111111111111100000000>

Rough UCCSD state:

-0.0000425937+0.0000000000j |00111111111111111100000011> +
-0.0000425937+0.0000000000j |00111111111111111100001100> +
-0.0000425937+0.0000000000j |00111111111111111100110000> +
-0.0000557415+0.0000000000j |00111111111111111111000000> +
-0.0000572857+0.0000000000j |01101111111111111100000011> +
-0.0000572857+0.0000000000j |01101111111111111100001100> +
-0.0000572857+0.0000000000j |01101111111111111100110000> +
 0.0000402165+0.0000000000j |01101111111111111111000000> +
-0.0000786709+0.0000000000j |01110111111111111110000010> +
-0.0000065843+0.0000000000j |01110111111111111110001000> +
-0.0000043301+0.0000000000j |01110111111111111110100000> +
-0.0000004279+0.0000000000j |01111011111111111100010010> +
 0.0000012500+0.0000000000j |01111011111111111100011000> +
 0.0000004279+0.0000000000j |01111011111111111100100001> +
-0.0000012500+0.0000000000j |01111011111111111100100100> +
 0.0000511531+0.0000000000j |01111

# Taper the Hamiltonian symmetries

In [54]:
from symred.S3_projection import QubitTapering

taper_hamiltonian = QubitTapering(ham)

print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
print('The symmetry generators are\n')
print(taper_hamiltonian.symmetry_generators)
print('\nand are rotated onto the single-qubit Pauli operators\n')
print(taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis())
print('\nvia a sequence of Clifford pi/2 rotations\n')
print(taper_hamiltonian.stabilizers.stabilizer_rotations)

We are able to taper 2 qubits from the Hamiltonian.

The symmetry generators are

1+0j ZIZIZIZIZIZIZIZIZIZIZIZIZI +
1+0j IZIZIZIZIZIZIZIZIZIZIZIZIZ

and are rotated onto the single-qubit Pauli operators

-1+0j IXIIIIIIIIIIIIIIIIIIIIIIII +
-1+0j XIIIIIIIIIIIIIIIIIIIIIIIII

via a sequence of Clifford pi/2 rotations

[('YIZIZIZIZIZIZIZIZIZIZIZIZI', None), ('IYIZIZIZIZIZIZIZIZIZIZIZIZ', None)]


In [55]:
taper_hamiltonian.stabilizers.update_sector(hf_array)
print(f'The symmetry sector corresponding with the single reference {hf_array} is {taper_hamiltonian.stabilizers.coeff_vec}')

The symmetry sector corresponding with the single reference [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0] is [-1 -1]


In [56]:
ham_tap = taper_hamiltonian.taper_it(ref_state=hf_array)
ucc_tap = taper_hamiltonian.taper_it(aux_operator=T_antiherm, ref_state=hf_array)
n_taper = taper_hamiltonian.n_taper
tapered_qubits   = taper_hamiltonian.stab_qubit_indices
untapered_qubits = taper_hamiltonian.free_qubit_indices
hf_tapered = taper_hamiltonian.tapered_ref_state

dashes = "------------------------------------------------"
print("Tapering information:")
print(dashes)
print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian')
print('The symmetry basis/sector is:') 
print(taper_hamiltonian.symmetry_generators)
print(f'The tapered Hartree-Fock state is', QuantumState([hf_tapered]))
print(dashes)

Tapering information:
------------------------------------------------
We are able to taper 2 qubits from the Hamiltonian
The symmetry basis/sector is:
-1 ZIZIZIZIZIZIZIZIZIZIZIZIZI +
-1 IZIZIZIZIZIZIZIZIZIZIZIZIZ
The tapered Hartree-Fock state is  1.0000000000 |111111111111111100000000>
------------------------------------------------


# Perform ADAPT-CS-VQE

In [ ]:
from symred.adapt_cs_vqe import ADAPT_CS_VQE
adapt = ADAPT_CS_VQE(ham_tap, ucc_tap, hf_tapered)
adapt.evaluation_method = 'qlm'
adapt.n_shots = None# 2**5
adapt.n_realizations = 1

In [21]:
stab_indices, energy, ansatz = adapt.adaptive_greedy_search(16, adaptive_from=15, depth=2, threshold=0.01, print_info=True)

--------------------------------------------
Using best gradient as termination criterion
--------------------------------------------

-------------------------------------------------
Searching for optimal 2-qubit contextual subspace
-------------------------------------------------

Testing contextual subspace 1 of 10 with stabilizer indices {2, 3, 4}
<!> The ansatz pool vanished under the noncontextual projection <!>

Testing contextual subspace 2 of 10 with stabilizer indices {1, 3, 4}
>>> Ansatz term #1 gradient = 0.17361795892304302.
>>> Ansatz term #2 gradient = 0.027865719848599143.
>>> Ansatz term #3 gradient = 0.0036505381475509593.
<!> The new ansatz term achieved the gradient threshold <!>

Testing contextual subspace 3 of 10 with stabilizer indices {1, 2, 4}
>>> Ansatz term #1 gradient = 0.17361795892304302.
>>> Ansatz term #2 gradient = 0.027865719848599146.
>>> Ansatz term #3 gradient = 0.0036505381475498214.
<!> The new ansatz term achieved the gradient threshold <!>



In [22]:
ham_cs = adapt.contextual_subspace_projection(stab_indices)
ham_cs = ObservableOp(ham_cs.symp_matrix, ham_cs.coeff_vec.real)
ref_state = hf_tapered[adapt.free_qubit_indices]

In [23]:
fci_energy = pyscf_obj.pyscf_fci.e_tot
error = abs(energy - fci_energy)
print('FCI energy:         ', fci_energy)
print('ADAPT-CS-VQE energy:', energy)
print()
print(f'Energy error = {error}')
print(f'Achieves chemical accuracy? {error < 0.0016}')

FCI energy:          -14.403655108067667
ADAPT-CS-VQE energy: -14.403265428353867

Energy error = 0.00038967971380010624
Achieves chemical accuracy? True


In [45]:
ham_cs.evaluation_method = 'qlm'
ham_cs.n_shots = None
ham_cs.ansatz_expectation(ansatz, ref_state)

-14.403265428353867

In [19]:
from qat.plugins import ScipyMinimizePlugin
from qat.qpus import LinAlg
from qat.dqs.vqe import VQE

shots      = 1024,
qpu_type   = LinAlg(),
opt_method = 'COBYLA',
opt_tol    = 1e-5,
maxiter    = 2000,
#zero_params = False,
#init_params= None,

hamiltonian = ham_cs.to_qlm
qc = ansatz.to_qlm_circuit(ref_state = hf_tapered, bind_params=False)
num_params = len(qc.get_variables())
theta0=np.zeros(num_params)

opt=ScipyMinimizePlugin(method=opt_method,
                              tol=opt_tol,
                              options={"maxiter": maxiter},
                              x0=theta0)

VQE(hamiltonian=hamiltonian, optimizer=opt, ansatz_routine=qc, theta0=theta0, qpu=LinAlg())


def fun(theta):
    qpu = LinAlg()
    prog = Program()
    reg = prog.qalloc(hamiltonian.nbqbits)
    prog.apply(qrout(theta), reg)
    circ = ansatz.to_qlm_circuit()
    res = qpu.submit(circ.to_job(job_type='OBS', observable=H_sp))
    return res.value

res = scipy.optimize.minimize(lambda theta: fun(theta), x0=theta_0,
                            method='COBYLA', options={'maxiter': 1000})

print("VQE energy =", res.x)
#qpu = optimizer | LinAlg()

#if isinstance(qpu_type, NoisyQProc):
#    job = qc.to_job(job_type="OBS", nbshots=shots, observable=hamiltonian)
#else:
#job = qc.to_job(job_type="OBS", observable=hamiltonian)
#result = qpu.submit(job)
#opt_data = result.meta_data['optimizer_data'] #stored as string... why!
#cut1 = opt_data[opt_data.find('nfev')+6:]
#cut2 = cut1[:cut1.find('\n')]
#nfev = int(cut2)

#return {'opt_value':result.value,
#        'opt_param':eval(result.meta_data['parameters']),
#        'nfev':nfev,
#        'value_trace':eval(result.meta_data['optimization_trace']), 
#        'param_trace':callback.param_config}

TypeError: 'ScipyMinimizePlugin' object is not callable

In [ ]:
qc.get_variables()

In [ ]:
ansatz.to_QuantumCircuit(bind_params=False).draw()

In [ ]:
from qat.lang.AQASM import *

def to_qlm_circuit(operator):
    
    def qlm_circuit(qc_qiskit):
        """Method for generating gate intructions to build CS-VQE circuit on Atos' QLM
        """
        n_qbits = qc_qiskit.num_qubits
        instructions=[]
        param_num=0
        for gate, q_pos, blank in qc_qiskit.data:
            var = gate.params
            if gate.name != 'barrier':
                if var != []:
                    var = [param_num%qc_qiskit.num_parameters]
                    param_num += 1
                instructions.append((gate.name.upper(), var, [q.index for q in q_pos]))

        qc_qlm_dict = { 'num_qubits':  n_qbits,
                        'num_params':  qc_qiskit.num_parameters,
                        'instructions':instructions}

        return n_qbits, qc_qiskit.num_parameters, instructions

    def build_qlm_circuit(num_sim_q, num_params, instructions):
        gate_dict = {'X':X,'H':H,'S':S,'SDG':S.dag(),'RZ':RZ,'CX':CNOT}
        prog = Program()
        qbits_reg = prog.qalloc(num_sim_q)
        params = [prog.new_var(float, "\\P{}".format(i)) for i in range(num_params)]

        for gate, var_num, q_pos in instructions:
            q_pos = [num_sim_q-p-1 for p in q_pos] #qubit ordering is reversed compared with Qiskit
            if gate == 'RZ':
                prog.apply(RZ(params[var_num[0]]), qbits_reg[q_pos[0]])
            elif gate == 'CX':
                prog.apply(CNOT, qbits_reg[q_pos[0]], qbits_reg[q_pos[1]])
            else:
                prog.apply(gate_dict[gate], qbits_reg[q_pos[0]])

        qc = prog.to_circ()

        return qc

    qc_qlm = build_qlm_circuit(*qlm_circuit(operator.to_QuantumCircuit(bind_params=False)))
    #bind_param = dict(zip(qc_qlm.get_variables(), ansatz.coeff_vec))
    return qc_qlm

In [ ]:
qc_qlm = to_qlm_circuit(ansatz)
qc_qlm.get_variables()

In [ ]:
from qat.qpus import get_default_qpu, LinAlg

# Create a job
job = qc_qlm.to_job(nbshots=10000)

In [ ]:
# Execute
result = get_default_qpu().submit(job)
for sample in result:
    print("State %s: probability %s +/- %s" % (sample.state, sample.probability, sample.err))

In [ ]:
from qat.core import Observable, Term

def construct_hamiltonian(ham_dict, num_sim_q):
    assert(len(list(ham_dict.keys())[0])==num_sim_q)
    pauli_terms = [Term(coeff, op, list(range(num_sim_q))) for op, coeff in ham_dict.items()]
    hamiltonian = Observable(num_sim_q,pauli_terms=pauli_terms)
    return hamiltonian

In [ ]:
obs = construct_hamiltonian(ham_cs.to_dictionary, ham_cs.n_qubits)

In [ ]:
# Create a job
job = qc_qlm.to_job(observable=obs, nbshots=100000)

# Execute
result = LinAlg().submit(job)
print("<O> = ", result.value)

In [ ]:
print(ham_cs)

In [ ]:
print(obs)

In [ ]:
print(qc_qlm.ops)

In [ ]:
ansatz.to_QuantumCircuit().draw()